In [2]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, LSTM, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, concatenate, Convolution1D
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from keras.callbacks import EarlyStopping
import codecs

import io
import random as rn
import tensorflow as tf

Using TensorFlow backend.


In [3]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [4]:

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

# load model
EMBEDDING_DIM = 300
file_loc = '../../dimensionalSM/data/crawl-300d-2M-subword.vec'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

# split train/test

earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)



Found 3437 unique tokens
../../dimensionalSM/data/crawl-300d-2M-subword.vec
G Word embeddings: 2000000
G Null word embeddings: 133


In [5]:
split = 7869

In [6]:
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    #net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    #out_text = Dropout(0.5)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model
model = text_model1()
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     1031400     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 256)     439296      embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 128)          164352      bidirectional_1[0][0]            
___________

In [7]:
# main function for for LOSO
def main(time):
    model = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model.fit( x_train_text[:split], 
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model.evaluate( x_train_text[split:], vad[split:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    weight = []
    for time in range(30):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('text_blstm_fasttext7869_no_dropout.csv', header=['time','v','a','d', 'ave'])



Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 44s 7ms/step - loss: 1.7840 - v_loss: 0.5551 - a_loss: 0.6193 - d_loss: 0.6094 - v_ccc: 0.4446 - a_ccc: 0.3805 - d_ccc: 0.3909 - val_loss: 1.9986 - val_v_loss: 0.7278 - val_a_loss: 0.6609 - val_d_loss: 0.6322 - val_v_ccc: 0.2769 - val_a_ccc: 0.3475 - val_d_ccc: 0.3771
Epoch 2/50
6295/6295 [==============================] - 41s 7ms/step - loss: 1.4003 - v_loss: 0.3810 - a_loss: 0.5148 - d_loss: 0.5047 - v_ccc: 0.6190 - a_ccc: 0.4851 - d_ccc: 0.4956 - val_loss: 1.9160 - val_v_loss: 0.6612 - val_a_loss: 0.6360 - val_d_loss: 0.6412 - val_v_ccc: 0.3431 - val_a_ccc: 0.3727 - val_d_ccc: 0.3682
Epoch 3/50
6295/6295 [==============================] - 41s 7ms/step - loss: 1.2513 - v_loss: 0.3281 - a_loss: 0.4662 - d_loss: 0.4569 - v_ccc: 0.6719 - a_ccc: 0.5336 - d_ccc: 0.5432 - val_loss: 1.9229 - val_v_loss: 0.6054 - val_a_loss: 0.6340 - val_d_loss: 0.7058 - val_v_ccc: 0.4006 - val_a_ccc: 0.3

Epoch 11/50
6295/6295 [==============================] - 42s 7ms/step - loss: 0.7086 - v_loss: 0.1751 - a_loss: 0.2729 - d_loss: 0.2603 - v_ccc: 0.8249 - a_ccc: 0.7271 - d_ccc: 0.7395 - val_loss: 1.8637 - val_v_loss: 0.5608 - val_a_loss: 0.6494 - val_d_loss: 0.6774 - val_v_ccc: 0.4466 - val_a_ccc: 0.3574 - val_d_ccc: 0.3323
Epoch 12/50
6295/6295 [==============================] - 43s 7ms/step - loss: 0.6726 - v_loss: 0.1713 - a_loss: 0.2527 - d_loss: 0.2486 - v_ccc: 0.8288 - a_ccc: 0.7472 - d_ccc: 0.7514 - val_loss: 1.8512 - val_v_loss: 0.5673 - val_a_loss: 0.6380 - val_d_loss: 0.6707 - val_v_ccc: 0.4402 - val_a_ccc: 0.3695 - val_d_ccc: 0.3390
Epoch 13/50
6295/6295 [==============================] - 40s 6ms/step - loss: 0.6398 - v_loss: 0.1583 - a_loss: 0.2443 - d_loss: 0.2374 - v_ccc: 0.8416 - a_ccc: 0.7558 - d_ccc: 0.7627 - val_loss: 1.8506 - val_v_loss: 0.5552 - val_a_loss: 0.6570 - val_d_loss: 0.6607 - val_v_ccc: 0.4521 - val_a_ccc: 0.3486 - val_d_ccc: 0.3487
Epoch 14/50
6295/6295 

6295/6295 [==============================] - 41s 6ms/step - loss: 1.4007 - v_loss: 0.3760 - a_loss: 0.5179 - d_loss: 0.5070 - v_ccc: 0.6237 - a_ccc: 0.4824 - d_ccc: 0.4931 - val_loss: 1.9025 - val_v_loss: 0.6125 - val_a_loss: 0.6234 - val_d_loss: 0.6851 - val_v_ccc: 0.3887 - val_a_ccc: 0.3855 - val_d_ccc: 0.3233
Epoch 3/50
6295/6295 [==============================] - 41s 6ms/step - loss: 1.2406 - v_loss: 0.3233 - a_loss: 0.4610 - d_loss: 0.4556 - v_ccc: 0.6765 - a_ccc: 0.5387 - d_ccc: 0.5442 - val_loss: 1.9142 - val_v_loss: 0.5910 - val_a_loss: 0.6622 - val_d_loss: 0.6790 - val_v_ccc: 0.4114 - val_a_ccc: 0.3451 - val_d_ccc: 0.3293
Epoch 4/50
6295/6295 [==============================] - 41s 6ms/step - loss: 1.1306 - v_loss: 0.2872 - a_loss: 0.4238 - d_loss: 0.4196 - v_ccc: 0.7128 - a_ccc: 0.5763 - d_ccc: 0.5803 - val_loss: 1.8388 - val_v_loss: 0.5709 - val_a_loss: 0.6262 - val_d_loss: 0.6644 - val_v_ccc: 0.4329 - val_a_ccc: 0.3832 - val_d_ccc: 0.3451
Epoch 5/50
6295/6295 [==============

Epoch 12/50
6295/6295 [==============================] - 40s 6ms/step - loss: 0.6606 - v_loss: 0.1644 - a_loss: 0.2460 - d_loss: 0.2507 - v_ccc: 0.8358 - a_ccc: 0.7542 - d_ccc: 0.7494 - val_loss: 1.9048 - val_v_loss: 0.5718 - val_a_loss: 0.6693 - val_d_loss: 0.6837 - val_v_ccc: 0.4342 - val_a_ccc: 0.3359 - val_d_ccc: 0.3251
Epoch 13/50
6295/6295 [==============================] - 40s 6ms/step - loss: 0.6433 - v_loss: 0.1586 - a_loss: 0.2415 - d_loss: 0.2433 - v_ccc: 0.8415 - a_ccc: 0.7584 - d_ccc: 0.7568 - val_loss: 1.9231 - val_v_loss: 0.5783 - val_a_loss: 0.6744 - val_d_loss: 0.6911 - val_v_ccc: 0.4280 - val_a_ccc: 0.3314 - val_d_ccc: 0.3175
Epoch 14/50
6295/6295 [==============================] - 41s 6ms/step - loss: 0.6091 - v_loss: 0.1535 - a_loss: 0.2279 - d_loss: 0.2276 - v_ccc: 0.8465 - a_ccc: 0.7721 - d_ccc: 0.7723 - val_loss: 1.8908 - val_v_loss: 0.5583 - val_a_loss: 0.6654 - val_d_loss: 0.6891 - val_v_ccc: 0.4495 - val_a_ccc: 0.3404 - val_d_ccc: 0.3194
Epoch 15/50
2170/2170 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

6295/6295 [==============================] - 42s 7ms/step - loss: 0.8019 - v_loss: 0.1964 - a_loss: 0.3066 - d_loss: 0.2990 - v_ccc: 0.8035 - a_ccc: 0.6937 - d_ccc: 0.7009 - val_loss: 1.9391 - val_v_loss: 0.5695 - val_a_loss: 0.6762 - val_d_loss: 0.7153 - val_v_ccc: 0.4375 - val_a_ccc: 0.3296 - val_d_ccc: 0.2938
Epoch 10/50
6295/6295 [==============================] - 42s 7ms/step - loss: 0.7538 - v_loss: 0.1840 - a_loss: 0.2822 - d_loss: 0.2878 - v_ccc: 0.8159 - a_ccc: 0.7179 - d_ccc: 0.7124 - val_loss: 1.8753 - val_v_loss: 0.5630 - val_a_loss: 0.6505 - val_d_loss: 0.6859 - val_v_ccc: 0.4453 - val_a_ccc: 0.3556 - val_d_ccc: 0.3238
Epoch 11/50
6295/6295 [==============================] - 42s 7ms/step - loss: 0.7092 - v_loss: 0.1759 - a_loss: 0.2704 - d_loss: 0.2624 - v_ccc: 0.8240 - a_ccc: 0.7295 - d_ccc: 0.7373 - val_loss: 1.9439 - val_v_loss: 0.5770 - val_a_loss: 0.6829 - val_d_loss: 0.7078 - val_v_ccc: 0.4311 - val_a_ccc: 0.3230 - val_d_ccc: 0.3020
Epoch 12/50
6295/6295 [===========

6295/6295 [==============================] - 42s 7ms/step - loss: 1.3850 - v_loss: 0.3750 - a_loss: 0.5142 - d_loss: 0.4960 - v_ccc: 0.6251 - a_ccc: 0.4861 - d_ccc: 0.5038 - val_loss: 1.9913 - val_v_loss: 0.6705 - val_a_loss: 0.6696 - val_d_loss: 0.6694 - val_v_ccc: 0.3310 - val_a_ccc: 0.3384 - val_d_ccc: 0.3392
Epoch 3/50
6295/6295 [==============================] - 42s 7ms/step - loss: 1.2523 - v_loss: 0.3302 - a_loss: 0.4724 - d_loss: 0.4492 - v_ccc: 0.6697 - a_ccc: 0.5274 - d_ccc: 0.5506 - val_loss: 1.8425 - val_v_loss: 0.5928 - val_a_loss: 0.6225 - val_d_loss: 0.6503 - val_v_ccc: 0.4119 - val_a_ccc: 0.3869 - val_d_ccc: 0.3587
Epoch 4/50
6295/6295 [==============================] - 42s 7ms/step - loss: 1.1564 - v_loss: 0.2961 - a_loss: 0.4358 - d_loss: 0.4239 - v_ccc: 0.7037 - a_ccc: 0.5639 - d_ccc: 0.5760 - val_loss: 1.8677 - val_v_loss: 0.5818 - val_a_loss: 0.6288 - val_d_loss: 0.6763 - val_v_ccc: 0.4197 - val_a_ccc: 0.3802 - val_d_ccc: 0.3324
Epoch 5/50
6295/6295 [==============

Epoch 14/50
6295/6295 [==============================] - 42s 7ms/step - loss: 0.6071 - v_loss: 0.1530 - a_loss: 0.2285 - d_loss: 0.2255 - v_ccc: 0.8470 - a_ccc: 0.7714 - d_ccc: 0.7745 - val_loss: 1.9280 - val_v_loss: 0.5536 - val_a_loss: 0.6993 - val_d_loss: 0.6977 - val_v_ccc: 0.4541 - val_a_ccc: 0.3063 - val_d_ccc: 0.3116
Epoch 15/50
6295/6295 [==============================] - 42s 7ms/step - loss: 0.5738 - v_loss: 0.1435 - a_loss: 0.2189 - d_loss: 0.2114 - v_ccc: 0.8564 - a_ccc: 0.7812 - d_ccc: 0.7886 - val_loss: 1.9301 - val_v_loss: 0.5645 - val_a_loss: 0.6645 - val_d_loss: 0.7234 - val_v_ccc: 0.4425 - val_a_ccc: 0.3420 - val_d_ccc: 0.2854
Epoch 16/50
2170/2170 [==============================] - 4s 2ms/step
[0.35397469997406006, 0.3482893109321594, 0.3898758292198181]
26
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 50s 8ms/step - loss: 1.8122 - v_loss: 0.5732 - a_loss: 0.6111 - d_loss: 0.6269 - v_ccc: 0.4266 - a_ccc: 0.3885

6295/6295 [==============================] - 43s 7ms/step - loss: 0.9013 - v_loss: 0.2225 - a_loss: 0.3375 - d_loss: 0.3410 - v_ccc: 0.7776 - a_ccc: 0.6622 - d_ccc: 0.6589 - val_loss: 1.8876 - val_v_loss: 0.6098 - val_a_loss: 0.6289 - val_d_loss: 0.6710 - val_v_ccc: 0.3957 - val_a_ccc: 0.3783 - val_d_ccc: 0.3384
Epoch 8/50
6295/6295 [==============================] - 43s 7ms/step - loss: 0.8454 - v_loss: 0.2077 - a_loss: 0.3168 - d_loss: 0.3216 - v_ccc: 0.7923 - a_ccc: 0.6837 - d_ccc: 0.6785 - val_loss: 1.8191 - val_v_loss: 0.5721 - val_a_loss: 0.6226 - val_d_loss: 0.6483 - val_v_ccc: 0.4339 - val_a_ccc: 0.3844 - val_d_ccc: 0.3626
Epoch 9/50
6295/6295 [==============================] - 42s 7ms/step - loss: 0.7945 - v_loss: 0.1929 - a_loss: 0.2985 - d_loss: 0.3034 - v_ccc: 0.8074 - a_ccc: 0.7015 - d_ccc: 0.6967 - val_loss: 1.8695 - val_v_loss: 0.5843 - val_a_loss: 0.6412 - val_d_loss: 0.6649 - val_v_ccc: 0.4207 - val_a_ccc: 0.3640 - val_d_ccc: 0.3458
Epoch 10/50
6295/6295 [=============

Epoch 14/50
6295/6295 [==============================] - 41s 7ms/step - loss: 0.5924 - v_loss: 0.1482 - a_loss: 0.2243 - d_loss: 0.2206 - v_ccc: 0.8520 - a_ccc: 0.7761 - d_ccc: 0.7794 - val_loss: 1.8990 - val_v_loss: 0.5571 - val_a_loss: 0.6789 - val_d_loss: 0.6879 - val_v_ccc: 0.4512 - val_a_ccc: 0.3279 - val_d_ccc: 0.3219
Epoch 15/50
2170/2170 [==============================] - 5s 2ms/step
[0.3699503540992737, 0.38659265637397766, 0.3870440423488617]
29
